The singleton pattern makes sure that a given class has always only one living
instance in the application. This can be used, for example, when you want to restrict
a resource access to one and only one memory context in the process. For instance,
a database connector class can be a singleton that deals with synchronization and
manages its data in memory. It makes the assumption that no other instance is
interacting with the database in the meantime

## Semi-idiom way (not safe)

In [8]:
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance

In [9]:
print("Before init instance")
print("`Singleton`'s inner `_instance`: ", id(Singleton._instance))
print("-" * 10)
a = Singleton()
b = Singleton()
print("After init instance")
print("`Singleton`'s inner `_instance`: ", id(Singleton._instance))
print("`a`'s inner `_instance`: ", id(a._instance))
print("`b`'s inner `_instance`: '", id(b._instance))
print("`a`: ", id(a))
print("`b`: ", id(b))

Before init instance
`Singleton`'s inner `_instance`:  1694540944
----------
After init instance
`Singleton`'s inner `_instance`:  2159455008192
`a`'s inner `_instance`:  2159455008192
`b`'s inner `_instance`: ' 2159455008192
`a`:  2159455008192
`b`:  2159455008192


The problem starts when you try to subclass your base singleton class and create an instance of this new
subclass if you already created an instance of the base class

In [43]:
del Singleton
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance
class ConcreteClass(Singleton):
    pass

# They are the same instance, which is dangerous
print("`Singleton`: ", id(Singleton()))
print("`ConcreteClass`: ", id(ConcreteClass()))

`Singleton`:  2159455554696
`ConcreteClass`:  2159455554696


This may become even more problematic when you notice that this behavior is
affected by an instance creation order. Depending on your class usage order, you
may or may not get the same result.

In [44]:
del Singleton
class Singleton:
    _instance = None
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance
class ConcreteClass(Singleton):
    pass

# Changed the creation order, the result is different.
print("`ConcreteClass`: ", id(ConcreteClass()))
print("`Singleton`: ", id(Singleton()))

`ConcreteClass`:  2159455555256
`Singleton`:  2159455554920


## Safer way by adding dict

It is a lot safer to use a more advanced technique—metaclasses. By adding a dict for a metaclass, you can affect the creation of your custom
classes. This allows creating a reusable singleton code

In [45]:
del Singleton
class Singleton(object):
    _instances = { }
    def __new__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__new__(cls, *args, **kwargs)
        return cls._instances[cls]

By using this Singleton as a metaclass for your custom classes, you are able to get
singletons that are safe to subclass and independent of instance creation order:

In [46]:
class ConcreteClass(Singleton):
    pass
print("`Singleton`: ", id(Singleton()))
print("`ConcreteClass`: ", id(ConcreteClass()))

`Singleton`:  2159455009760
`ConcreteClass`:  2159455007128


In [47]:
del Singleton
class Singleton(object):
    _instances = {}
    def __new__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__new__(cls, *args, **kwargs)
        return cls._instances[cls]
class ConcreteClass(Singleton):
    pass
print("`ConcreteClass`: ", id(ConcreteClass()))
print("`Singleton`: ", id(Singleton()))

`ConcreteClass`:  2159455553800
`Singleton`:  2159455552120


## Another way to construct a Singleton

Another way to overcome the problem of trivial singleton implementation is to use
what Alex Martelli proposed. He came out with something similar in behavior to
singleton but completely different in structure. This is not a classical design pattern
coming from the GoF book, but it seems to be common among Python developers. It
is called `Borg` or `Monostate`.

The idea is quite simple. What really matters in the singleton pattern is not the
number of living instances a class has, but rather the fact that they all share the same
state at all times. So, Alex Martelli came up with a class that makes all instances of
the class share the same `__dict__`:

In [49]:
class Borg(object):
    _state = {}
    def __new__(cls, *args, **kwargs):
        ob = super().__new__(cls, *args, **kwargs)
        ob.__dict__ = cls._state
        return ob